In [ ]:
from esco import LocalDB
db = LocalDB()
skills_labels = [
        "collaborate with engineers",
        "deploy cloud resource",
        "design cloud architecture",
        "design cloud networks",
        "plan migration to cloud",
        "automate cloud tasks",
        "coordinate engineering teams",
        "design database in the cloud",
        "design for organisational complexity",
        "develop with cloud services",
        "do cloud refactoring",
    ]
skills = db.skills[db.skills.label.str.lower().isin(skills_labels)]
labels  = [l for labels in skills.allLabel for l in labels]

import spacy
nlp = spacy.load("en_core_web_trf")

In [ ]:
def find_root(doc):
    for prefix in ("", "to "):
        doc = nlp(prefix + doc.text)
        for token in doc:
            if token.dep_ == "ROOT" and token.pos_ == "VERB":
                return token
    return None

def find_obj(token):
    for child in token.children:
        if child.dep_ == "prep":
            return find_obj(child)
        if child.dep_ in ("dobj", "pobj", "nsubj"):
            return child
    return None

def find_compound(token):
    compounds = []
    if token:
        compounds.append(token.lemma_)
        for child in token.children:
            if child.dep_ == "compound":
                compounds.append(child.lemma_)
    return " ".join(reversed(compounds))  # Combine compounds into a single string
comp_list = []
docs = nlp.pipe(labels)
for doc in docs:
    verb = None
    for token in doc:
        verb = find_root(doc)
    if not verb:
        print("****missing verb", doc.text)
        continue
    obj = find_obj(verb)
    compound_str = find_compound(obj)
    comp_list.append(compound_str)
    print(verb.lemma_, compound_str, f"-{doc.text}-")


In [ ]:

# Funzione per estrarre competenze potenziali
def extract_potential_skills(text):
    doc = nlp(text)
    chunks = []
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ in ['dobj', 'pobj'] and chunk.root.head.pos_ == 'VERB':
            verb = chunk.root.head
            start = verb.i
            end = chunk.end
            chunks.append(doc[start:end])
    return chunks

# Funzione per valutare la rilevanza di un chunk come competenza
def evaluate_chunk_as_skill(chunk, known_skills):
    chunk_text = chunk.text.lower()
    for skill in known_skills:
        if skill in chunk_text:
            return True
    return False

text = "During my internship, I wrote code to integrate cloud services and cloud networks."

potential_skills = extract_potential_skills(text)
for chunk in potential_skills:
    if evaluate_chunk_as_skill(chunk, comp_list):
        print(f"Potential skill identified: {chunk}")

```
Potential skill identified: wrote code
Potential skill identified: integrate cloud services
```


In [ ]:
test_text = [
    "I am an experienced IT professional with a strong background in cloud computing and software development. ",
    "I specialize in planning and executing cloud migrations, designing cloud architectures, and automating cloud tasks. ",
    "My skills include creating cloud network, managing cloud resource, and developing cloud applications. ",
    "I am also adept to designing cloud environments for large organizations, ensuring they are scalable and efficient."
]
for text in test_text:
    potential_skills = extract_potential_skills(text)
    for chunk in potential_skills:
        if evaluate_chunk_as_skill(chunk, comp_list):
            print(f"Potential skill identified: {chunk}")

```
Potential skill identified: executing cloud migrations
Potential skill identified: designing cloud architectures
Potential skill identified: automating cloud tasks
Potential skill identified: creating cloud network
Potential skill identified: managing cloud resource
Potential skill identified: developing cloud applications
Potential skill identified: designing cloud environments
```